# Recreating the code

# Environmment related code

In [144]:
import numpy as np
import random

class DisasterZoneEnv:
    """
    A 2D grid environment for a drone exploring a disaster zone.

    Legend (internally stored as integers):
        0 -> Empty cell
        1 -> Obstacle
        2 -> Survivor
        3 -> Recharging Point

    'D' in a scenario array indicates the drone's starting position (only in predefined scenarios).
    """

    def __init__(self, 
                 width=8, 
                 height=8, 
                 num_obstacles=5, 
                 num_survivors=3, 
                 num_resources=2, 
                 initial_energy=20, 
                 dynamic=False, 
                 predefined_grid=None):
        """
        Initialize the environment. If a predefined grid is provided, use it; otherwise, generate randomly.

        :param width: Width of the grid.
        :param height: Height of the grid.
        :param num_obstacles: Number of obstacles.
        :param num_survivors: Number of survivors.
        :param num_resources: Number of recharging points.
        :param initial_energy: Initial energy of the drone.
        :param dynamic: Whether the environment is dynamic.
        :param predefined_grid: A predefined grid layout (optional, can contain 'D').
        """
        self.width = width
        self.height = height
        self.num_obstacles = num_obstacles
        self.num_survivors = num_survivors
        self.num_resources = num_resources
        self.initial_energy = initial_energy
        self.dynamic = dynamic
        self.energy = initial_energy
        self.dynamic_changes = 0  # Counter for dynamic changes

        # If predefined_grid is given, use that scenario
        if predefined_grid is not None:
            self.reset_with_scenario(predefined_grid)
        else:
            self.reset()

    def _get_random_empty_cell(self):
        """
        Finds a random empty cell in the grid (cell == 0).
        """
        while True:
            x = random.randint(0, self.height - 1)
            y = random.randint(0, self.width - 1)
            if self.grid[x, y] == 0:  # Ensure the cell is empty
                return x, y

    def reset(self):
        """
        Resets the environment to a new random configuration.
        """
        self.grid = np.zeros((self.height, self.width), dtype=int)

        # Place obstacles
        for _ in range(self.num_obstacles):
            x, y = self._get_random_empty_cell()
            self.grid[x, y] = 1

        # Place survivors
        for _ in range(self.num_survivors):
            x, y = self._get_random_empty_cell()
            self.grid[x, y] = 2

        # Place resources
        for _ in range(self.num_resources):
            x, y = self._get_random_empty_cell()
            self.grid[x, y] = 3

        # Random start for the drone
        self.drone_x, self.drone_y = self._get_random_empty_cell()
        self.energy = self.initial_energy

    def reset_with_scenario(self, scenario):
        """
        Loads a predefined grid (possibly containing 'D') into the environment.
        We convert that scenario into an integer grid internally.
        """
        # Ensure scenario is at least a NumPy array
        scenario = np.array(scenario, dtype=object)  # Force an object array (handles mixed int/'D')

        # Prepare an integer grid of the same shape
        self.height, self.width = scenario.shape
        self.grid = np.zeros((self.height, self.width), dtype=int)

        drone_positions = []

        # Convert each cell from scenario into our integer grid
        for x in range(self.height):
            for y in range(self.width):
                cell_value = scenario[x, y]
                if cell_value == 'D':
                    # Mark drone position
                    drone_positions.append((x, y))
                    self.grid[x, y] = 0  # The drone is effectively on an empty cell
                else:
                    # Convert string or int to int properly
                    # If cell_value is already an int, this will do nothing special.
                    # If it's a string like '1', '2', '3', or '0', convert to int.
                    self.grid[x, y] = int(cell_value)

        if len(drone_positions) != 1:
            raise ValueError("Scenario must contain exactly one 'D' for the drone's starting position.")

        self.drone_x, self.drone_y = drone_positions[0]
        self.energy = self.initial_energy

    def apply_dynamic_changes(self, step_count):
        """
        Applies dynamic changes to the environment if self.dynamic is True.

        :param step_count: Current step count of the simulation.
        """
        if self.dynamic:
            # Example dynamic behavior:
            # 1) Add a new obstacle every 5 steps
            if step_count % 5 == 0:
                x, y = self._get_random_empty_cell()
                self.grid[x, y] = 1
                self.dynamic_changes += 1

            # 2) Move all survivors every 3 steps
            if step_count % 3 == 0:
                survivor_positions = [(x, y) for x in range(self.height) 
                                      for y in range(self.width) if self.grid[x, y] == 2]
                for (sx, sy) in survivor_positions:
                    self.grid[sx, sy] = 0  # Remove old survivor
                    new_x, new_y = self._get_random_empty_cell()
                    self.grid[new_x, new_y] = 2

    def render(self):
        """
        Renders the grid for visualization in a text-based manner.
        """
        grid_copy = self.grid.astype(str)
        grid_copy[grid_copy == '0'] = '.'
        grid_copy[grid_copy == '1'] = '#'
        grid_copy[grid_copy == '2'] = 'S'
        grid_copy[grid_copy == '3'] = 'R'

        # Mark the drone in the visualization
        grid_copy[self.drone_x, self.drone_y] = 'D'

        for row in grid_copy:
            print(" ".join(row))
        print(f"Energy: {self.energy}\n")


# Agent related code

In [145]:
import heapq

class DijkstraAgent:
    """
    An agent that uses Dijkstra's algorithm to navigate the DisasterZoneEnv.
    """

    def __init__(self, env):
        self.env = env
        self.steps_taken = 0
        self.survivors_rescued = 0
        self.resources_collected = 0

    def dijkstra(self, grid, start, target):
        """
        Finds the shortest path from start to target using Dijkstra's algorithm.
        """
        directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
        pq = [(0, start)]  # Priority queue (distance, position)
        distances = {start: 0}
        previous = {start: None}

        while pq:
            current_distance, current_position = heapq.heappop(pq)
            if current_position == target:
                path = []
                while current_position:
                    path.append(current_position)
                    current_position = previous[current_position]
                path.reverse()
                return path, current_distance

            for dx, dy in directions:
                neighbor = (current_position[0] + dx, current_position[1] + dy)
                if 0 <= neighbor[0] < grid.shape[0] and 0 <= neighbor[1] < grid.shape[1]:
                    if grid[neighbor[0], neighbor[1]] != 1:  # Ignore obstacles
                        new_distance = current_distance + 1
                        if neighbor not in distances or new_distance < distances[neighbor]:
                            distances[neighbor] = new_distance
                            previous[neighbor] = current_position
                            heapq.heappush(pq, (new_distance, neighbor))
        # Debugging: Print if no path is found
        print(f"No path found from {start} to {target}")
        return [], float('inf')  # No valid path

    def find_closest_target(self, target_type):
        start = (self.env.drone_x, self.env.drone_y)
        targets = [(x, y) for x in range(self.env.height) for y in range(self.env.width) if self.env.grid[x, y] == target_type]

        # Debugging: Print all targets found
        print(f"All targets of type {target_type}: {targets}")

        if not targets:
            print(f"No targets of type {target_type} found!")
            return None, None, float('inf')  # No targets

        shortest_path, closest_target, shortest_distance = None, None, float('inf')
        for target in targets:
            path, distance = self.dijkstra(self.env.grid, start, target)
            if distance < shortest_distance:
                shortest_path, closest_target, shortest_distance = path, target, distance

        print(f"Closest target of type {target_type} found at {closest_target} with distance {shortest_distance}")

        # Debugging: Print the closest target
        print(f"Closest target of type {target_type} found at {closest_target} with distance {shortest_distance}")
        return shortest_path, closest_target, shortest_distance

    def execute(self):
        """
        Executes the agent's logic in the environment.
        """
        step_count = 0
        while self.env.energy > 0:
            path, target_position, _ = self.find_closest_target(2)  # Look for survivors first
            if not path:
                path, target_position, _ = self.find_closest_target(3)  # Then look for resources

            if not path:
                print("No reachable targets. Stopping.")
                break

            for step in path[1:]:
                self.env.drone_x, self.env.drone_y = step
                self.steps_taken += 1
                step_count += 1

                if self.env.dynamic:
                    self.env.apply_dynamic_changes(step_count)

                self.env.energy -= 1
                if self.env.energy <= 0:
                    break

            if target_position:
                if self.env.grid[target_position[0], target_position[1]] == 2:
                    self.survivors_rescued += 1
                elif self.env.grid[target_position[0], target_position[1]] == 3:
                    self.resources_collected += 1
                self.env.grid[target_position[0], target_position[1]] = 0


# Tester related code

In [146]:
import time
import pandas as pd

class Tester:
    """
    Handles scenario-based testing and metric collection for agents.
    """

    def __init__(self, agent_class, scenarios, env_params=None):
        """
        :param agent_class: The class of the agent (e.g., DijkstraAgent).
        :param scenarios: A list of predefined grids (each may contain 'D').
        :param env_params: Dictionary with environment parameters like {"initial_energy": 20, ...}
        """
        self.agent_class = agent_class
        self.scenarios = scenarios
        self.env_params = env_params if env_params is not None else {}
        self.results = []

    def run_scenario(self, scenario, scenario_id):

        # Create environment with the specified scenario
        env = DisasterZoneEnv(
            predefined_grid=scenario, 
            initial_energy=self.env_params.get("initial_energy", 20),
            dynamic=self.env_params.get("dynamic", False)
        )
        agent = self.agent_class(env)

        start_time = time.time()
        agent.execute()
        end_time = time.time()

        computation_time = end_time - start_time

        # Record results
        self.results.append({
            "Agent Name": self.agent_class.__name__,
            "Scenario ID": scenario_id,
            "Steps Taken": agent.steps_taken,
            "Survivors Rescued": agent.survivors_rescued,
            "Resources Collected": agent.resources_collected,
            "Energy Used": self.env_params.get("initial_energy", 20) - env.energy,
            "Computation Time (s)": computation_time
        })

    def run_all_scenarios(self):
        """
        Run the agent on all given scenarios.
        """
        for scenario_id, scenario in enumerate(self.scenarios):
            print(f"\n--- Running scenario {scenario_id} ---")
            self.run_scenario(scenario, scenario_id)

    def save_results(self, filename="results.csv"):
        """
        Save the testing results to a CSV file.
        """
        df = pd.DataFrame(self.results)
        df.to_csv(filename, index=False)
        print(f"Results saved to {filename}")


# Predefined scenarios for testing

In [147]:
import numpy as np

# Scenario 1: Simple Layout
scenario_1 = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 2, 0, 1, 0, 0, 3, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 2, 0, 0],
    [3, 0, 0, 0, 0, 0, 0, 'D']
], dtype=object)

# Scenario 2: Obstacle Maze
scenario_2 = np.array([
    [0, 1, 1, 1, 0, 0, 0, 0],
    [0, 1, 2, 1, 0, 1, 1, 0],
    [0, 1, 0, 1, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 1, 0, 3],
    [0, 1, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0, 0, 2, 0],
    ['D', 1, 0, 0, 0, 0, 0, 0]
], dtype=object)

# Scenario 3
scenario_3 = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 3, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 3, 0, 0, 0, 0],
    ['D', 0, 0, 0, 0, 0, 0, 0]
], dtype=object)

# Scenario 4
scenario_4 = np.array([
    [0, 0, 0, 0, 3, 0, 3, 0],
    [0, 3, 0, 3, 0, 3, 0, 3],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [3, 0, 2, 0, 3, 0, 2, 0],
    [0, 0, 0, 3, 0, 0, 0, 3],
    [0, 3, 0, 0, 0, 3, 0, 0],
    [0, 0, 0, 3, 0, 2, 0, 3],
    ['D', 0, 0, 0, 0, 0, 0, 0]
], dtype=object)

# Scenario 5
scenario_5 = np.array([
    [1, 1, 1, 1, 1, 1, 1, 1],
    [1, 0, 0, 0, 0, 0, 0, 1],
    [1, 0, 1, 1, 1, 1, 0, 1],
    [1, 0, 1, 0, 0, 1, 0, 1],
    [1, 0, 1, 0, 3, 1, 0, 1],
    [1, 0, 1, 0, 0, 1, 0, 1],
    [1, 0, 0, 2, 0, 0, 0, 1],
    [1, 'D', 1, 1, 1, 1, 1, 1]
], dtype=object)

# Scenario 6
scenario_6 = np.array([
    [0, 0, 0, 0, 2, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 3, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 3, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 2, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 3, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    ['D', 0, 0, 0, 0, 0, 0, 0, 0, 0]
], dtype=object)

# Scenario 7
scenario_7 = np.array([
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 0, 0, 0],
    [0, 2, 2, 1, 1, 0, 0, 0],
    [0, 0, 0, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 3, 3, 0, 0],
    ['D', 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0]
], dtype=object)

# Scenario 9 (There is no scenario_8 in your original code)
scenario_9 = np.array([
    [0, 0, 0, 2, 2, 0, 0, 0],
    [0, 0, 0, 2, 2, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 0, 0, 0],
    [0, 3, 0, 1, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0],
    ['D', 0, 0, 0, 0, 0, 0, 0]
], dtype=object)

# A small test scenario
test_scenario = np.array([
    [0, 0, 0, 0],
    [0, 2, 0, 0],
    [0, 0, 0, 0],
    ['D', 0, 0, 0]
], dtype=object)


# Running the code

In [151]:
# List of scenarios to test
scenarios = [
    scenario_1,
    scenario_2,
    scenario_3,
    scenario_4,
    scenario_5,
    scenario_6,
    scenario_7,
    scenario_9
]

# Environment parameters
env_params = {
    "initial_energy": 20,  # Drone's initial energy
    "dynamic": False       # Set to False for static env
}

# Create a tester
tester = Tester(DijkstraAgent, scenarios, env_params=env_params)

# Run all scenarios
tester.run_all_scenarios()

# Save results to CSV
tester.save_results("results_dynamic.csv")



--- Running scenario 0 ---
All targets of type 2: [(2, 1), (6, 5)]
Closest target of type 2 found at (6, 5) with distance 3
Closest target of type 2 found at (6, 5) with distance 3
All targets of type 2: [(0, 2), (1, 1)]
Closest target of type 2 found at (0, 2) with distance 9
Closest target of type 2 found at (0, 2) with distance 9
All targets of type 2: [(4, 0), (6, 0)]
Closest target of type 2 found at (4, 0) with distance 6
Closest target of type 2 found at (4, 0) with distance 6
All targets of type 2: [(2, 2), (7, 4)]
Closest target of type 2 found at (2, 2) with distance 4
Closest target of type 2 found at (2, 2) with distance 4

--- Running scenario 1 ---
All targets of type 2: [(1, 2), (6, 6)]
Closest target of type 2 found at (1, 2) with distance 8
Closest target of type 2 found at (1, 2) with distance 8
All targets of type 2: [(4, 6), (7, 3)]
Closest target of type 2 found at (7, 3) with distance 7
Closest target of type 2 found at (7, 3) with distance 7
All targets of type 